# Importing Data On HealthCare Costs

*Make sure to open terminal and cd to your working directory*

In [3]:
import pandas as pd

## Import heart attack data

In [5]:
ha_costs_df = pd.read_csv('data/mmd_heart_attack_data.csv')

*Examine the first 5 rows and the last five rows of the TN heart attack  data.*

In [6]:
ha_costs_df.head(5)

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
0,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1001,Autauga County,ALABAMA,Urban,All,undefined,40470
1,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1003,Baldwin County,ALABAMA,Rural,All,undefined,36615
2,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1005,Barbour County,ALABAMA,Rural,All,undefined,46509
3,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1007,Bibb County,ALABAMA,Urban,All,undefined,42949
4,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1009,Blount County,ALABAMA,Urban,All,undefined,50067


In [7]:
ha_costs_df.tail(5)

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
2651,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72097,Mayaguez Municipio,PUERTO RICO,Urban,All,undefined,28128
2652,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72113,Ponce Municipio,PUERTO RICO,Urban,All,undefined,26403
2653,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72125,San German Municipio,PUERTO RICO,Urban,All,undefined,26398
2654,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72127,San Juan Municipio,PUERTO RICO,Urban,All,undefined,36599
2655,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72139,Trujillo Alto Municipio,PUERTO RICO,Urban,All,undefined,26676


*Check the data types of each column.*

In [8]:
ha_costs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656 entries, 0 to 2655
Data columns (total 17 columns):
year                   2656 non-null int64
geography              2656 non-null object
measure                2656 non-null object
adjustment             2656 non-null object
analysis               2656 non-null object
domain                 2656 non-null object
condition              2656 non-null object
primary_sex            2656 non-null object
primary_age            2656 non-null object
primary_dual           2656 non-null object
fips                   2656 non-null int64
county                 2656 non-null object
state                  2656 non-null object
urban                  2656 non-null object
primary_race           2656 non-null object
primary_denominator    2656 non-null object
analysis_value         2656 non-null int64
dtypes: int64(3), object(14)
memory usage: 352.8+ KB


### Keep only the rows where facilities are in TN

In [9]:
TN_heart = ha_costs_df.query('state == "TENNESSEE"')

*Print the dimensions of the data.*

In [10]:
TN_heart.shape

(91, 17)

*See how many TN counties are represented in the heart attack costs data.*

In [11]:
TN_heart.county.nunique()

91

*See how many counties are classified as Urban and how many as Rural.*

In [12]:
Urb_heart = TN_heart.query('urban =="Urban"')

In [13]:
Urb_heart.shape

(38, 17)

In [14]:
Rur_heart = TN_heart.query('urban =="Rural"')

In [15]:
Rur_heart.shape

(53, 17)

*Examine the distribution of costs.*

In [16]:
TN_heart.describe()

,year,fips,analysis_value
count,91.0,91.000000,91.000000
mean,2017.0,47093.615385,45762.989011
std,0.0,55.338708,6574.670462
min,2017.0,47001.000000,30831.000000
25%,2017.0,47046.000000,41105.500000
50%,2017.0,47093.000000,45384.000000
75%,2017.0,47142.000000,49698.500000
max,2017.0,47189.000000,62641.000000


In [17]:
Urb_heart.describe()

,year,fips,analysis_value
count,38.0,38.000000,38.000000
mean,2017.0,47103.789474,45748.921053
std,0.0,60.904071,7109.788921
min,2017.0,47001.000000,32279.000000
25%,2017.0,47049.500000,40452.750000
50%,2017.0,47112.000000,45496.500000
75%,2017.0,47160.500000,50734.500000
max,2017.0,47189.000000,61541.000000


In [18]:
Rur_heart.describe()

,year,fips,analysis_value
count,53.0,53.000000,53.000000
mean,2017.0,47086.320755,45773.075472
std,0.0,50.309723,6232.724885
min,2017.0,47003.000000,30831.000000
25%,2017.0,47045.000000,41240.000000
50%,2017.0,47085.000000,45384.000000
75%,2017.0,47123.000000,48998.000000
max,2017.0,47185.000000,62641.000000


**There does not seem to be a significant difference in heart attack costs between urban and rural counties at first glance.**

## Import Cancer Data

In [19]:
cancer_costs_df = pd.read_csv("data/mmd_cancer_data.csv")

*Look at the head and tail of the DataFrame*

In [20]:
cancer_costs_df.head()

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
0,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,1001,Autauga County,ALABAMA,Urban,All,undefined,19293
1,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,1003,Baldwin County,ALABAMA,Rural,All,undefined,17151
2,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,1005,Barbour County,ALABAMA,Rural,All,undefined,19469
3,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,1007,Bibb County,ALABAMA,Urban,All,undefined,17234
4,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,1009,Blount County,ALABAMA,Urban,All,undefined,20317


In [21]:
cancer_costs_df.tail()

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
3160,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,72145,Vega Baja Municipio,PUERTO RICO,Urban,All,undefined,7364
3161,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,72147,Vieques Municipio,PUERTO RICO,Rural,All,undefined,9987
3162,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,72149,Villalba Municipio,PUERTO RICO,Urban,All,undefined,24127
3163,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,72151,Yabucoa Municipio,PUERTO RICO,Urban,All,undefined,8008
3164,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,"Cancer, Colorectal, Breast, Prostate, Lung",All,All,Dual & non-dual,72153,Yauco Municipio,PUERTO RICO,Urban,All,undefined,11171


*Print the shape of the DataFrame.*

In [22]:
cancer_costs_df.shape

(3165, 17)

### Keep only the rows where facilities are in TN.

In [23]:
TN_cancer = cancer_costs_df.query('state == "TENNESSEE"')

*Print the dimensions of the data.*

In [24]:
TN_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 2396 to 2490
Data columns (total 17 columns):
year                   95 non-null int64
geography              95 non-null object
measure                95 non-null object
adjustment             95 non-null object
analysis               95 non-null object
domain                 95 non-null object
condition              95 non-null object
primary_sex            95 non-null object
primary_age            95 non-null object
primary_dual           95 non-null object
fips                   95 non-null int64
county                 95 non-null object
state                  95 non-null object
urban                  95 non-null object
primary_race           95 non-null object
primary_denominator    95 non-null object
analysis_value         95 non-null int64
dtypes: int64(3), object(14)
memory usage: 13.4+ KB


*See how many TN counties are represented in the cancer costs data.*

In [25]:
TN_cancer.county.nunique()

95

*Find which counties are missing from one of the datasets.*

In [26]:
list(set(TN_cancer.county)-set(TN_heart.county))

['Moore County', 'Hancock County', 'Pickett County', 'Van Buren County']

In [27]:
#The above method assumes that we aren't missing any counties from the cancer set that are in the heart set. To remedy this:

cancer_counties = list(TN_cancer.county.unique())
ha_counties = list(TN_heart.county.unique())

list(set(cancer_counties).difference(ha_counties))

['Van Buren County', 'Pickett County', 'Moore County', 'Hancock County']

*Examine the distribution in costs between urban and rural counties.*

In [28]:
Urb_cancer = TN_cancer.query('urban == "Urban"')

In [29]:
Rur_cancer = TN_cancer.query('urban == "Rural"')

In [30]:
Urb_cancer.describe()

,year,fips,analysis_value
count,38.0,38.000000,38.000000
mean,2017.0,47103.789474,19638.605263
std,0.0,60.904071,2712.218435
min,2017.0,47001.000000,15454.000000
25%,2017.0,47049.500000,17674.750000
50%,2017.0,47112.000000,19203.500000
75%,2017.0,47160.500000,20556.750000
max,2017.0,47189.000000,27740.000000


In [31]:
Rur_cancer.describe()

,year,fips,analysis_value
count,57.0,57.000000,57.000000
mean,2017.0,47089.140351,19837.421053
std,0.0,50.642810,2255.416100
min,2017.0,47003.000000,15531.000000
25%,2017.0,47049.000000,18328.000000
50%,2017.0,47087.000000,19633.000000
75%,2017.0,47129.000000,21315.000000
max,2017.0,47185.000000,25909.000000


In [32]:
TN_cancer.describe()

,year,fips,analysis_value
count,95.0,95.000000,95.000000
mean,2017.0,47095.000000,19757.894737
std,0.0,55.136195,2436.304533
min,2017.0,47001.000000,15454.000000
25%,2017.0,47048.000000,18103.500000
50%,2017.0,47095.000000,19563.000000
75%,2017.0,47142.000000,21153.000000
max,2017.0,47189.000000,27740.000000


**There does not seem to be a significant difference between cancer costs at first glance between rural and urban counties in TN.**

## Import the IRS income data.

In [33]:
income_df = pd.read_csv("data/irs_county_2016.csv")

### Keep only the income data that pertains to TN.

In [34]:
TN_income = income_df.query('STATE == "TN"')

*Look at the head and tail of the DataFrame.*

In [35]:
TN_income.head()

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
19752,47,TN,0,Tennessee,1,39580,22400,14440,980,25390,...,6760,17669,200,287,0,0,3970,7167,16170,59567
19753,47,TN,0,Tennessee,2,439770,345410,39920,49300,188490,...,109250,62045,0,0,0,0,37380,18477,366150,366510
19754,47,TN,0,Tennessee,3,717430,398720,115040,189290,341750,...,449870,461060,0,0,0,0,63270,70761,635450,1870568
19755,47,TN,0,Tennessee,4,759340,355970,217710,159890,347430,...,629080,1669328,110,100,0,0,104290,176690,646420,1783890
19756,47,TN,0,Tennessee,5,396810,128390,215890,42240,198820,...,382580,2135577,30,29,0,0,87440,220372,302390,849648


In [36]:
TN_income.tail()

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
20515,47,TN,189,Wilson County,4,14400,7130,4050,2610,6520,...,12340,34654,0,0,0,0,2090,4150,12140,31712
20516,47,TN,189,Wilson County,5,9330,2890,5160,960,4600,...,9010,50591,0,0,0,0,2080,5356,7100,19970
20517,47,TN,189,Wilson County,6,7160,1060,5630,380,3560,...,7110,61964,0,0,0,0,1710,5743,5290,17041
20518,47,TN,189,Wilson County,7,10350,780,9260,240,5330,...,10330,190229,60,13,0,0,3810,19921,6270,24591
20519,47,TN,189,Wilson County,8,2370,180,2130,50,1700,...,2370,230759,1160,1817,1080,3775,1260,39698,880,9411


*Look at the shape of the DataFrame.*

In [37]:
TN_income.shape

(768, 148)

*Keep only the following columns:  `['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']`*

In [38]:
TN_income_short = TN_income[['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']]

*Rename those columns: `['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']`*

In [39]:
TN_income_short.columns = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']

*Create `income_dict`, a dictionary that uses the numerical codes in the income bucket column as keys and the matching descriptions*

In [40]:
TN_income_short.income_bucket.unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

In [41]:
#Total is anything with Tennessee in the county column
#1 = 'Under $1'
# and going up from there
#use syntax dict({1: 'Geeks', 2: 'For', 3:'Geeks'}) 
#need to change the 'income_bucket' column in cases where TN is the county
#then I can use the dictionary
#Maybe I can split the df in two, make a dictionary and change the first 8 rows to Total, change the rest according to the dict, and then paste together

In [61]:
TN_income_sc = TN_income_short.copy()

Create `income_dict`, a dictionary that uses the numerical codes in the income bucket column as keys and the matching descriptions

In [159]:
income_dict = dict([(1, 'Under $1'), 
                    (2,'Between 1 and $10,000'), 
                    (3,'Between 10,000 and $25,000'),
                    (4,'Between 25,000 and $50,000'), 
                    (5,'Between 50,000 and $75,000'), 
                    (6,'Between 75,000 and $100,000'), 
                    (7,'Between 100,000 and $200,000'), 
                    (8,'$200,000 or more')])

Pass the dictionary as an argument to the pandas `replace()` method to change the `income_bucket` column so that it uses descriptive text instead of the numeric code.

In [163]:
TN_income_sc = TN_income_sc.replace({"income_bucket": income_dict})

In [164]:
TN_income_sc.head()

,state,county,income_bucket,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
19752,TN,Tennessee,Under $1,39580,22400,14440,980,60360,8230,19090,-2747555,29090,-2710342,90,348
19753,TN,Tennessee,"Between 1 and $10,000",439770,345410,39920,49300,443540,108380,74190,2366417,439780,2441687,4830,12132
19754,TN,Tennessee,"Between 10,000 and $25,000",717430,398720,115040,189290,1269420,472350,141070,12289681,717430,12486175,19660,58771
19755,TN,Tennessee,"Between 25,000 and $50,000",759340,355970,217710,159890,1486710,512170,154020,27370414,759340,27694026,19120,57012
19756,TN,Tennessee,"Between 50,000 and $75,000",396810,128390,215890,42240,881020,268500,106740,24350682,396810,24651803,10630,33894


Use the pandas `groupby()` method to group the data by county and get the `sum()` of all numeric columns for that county. Be sure to also `reset_index()` so that our aggregated data is re-indexed to begin at 0. Save this as a DataFrame called `income_county_agg`, and look at the first few rows.

In [167]:
income_county_agg = TN_income_sc.groupby(['county']).sum().reset_index()

In [169]:
income_county_agg.head()

,county,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
0,Anderson County,34290,14990,14030,4550,65950,19620,9670,1807309,34140,1830482,700,2452
1,Bedford County,20920,8600,8010,3910,43550,15790,4430,971152,20840,985909,410,1023
2,Benton County,6610,2660,2840,1000,13020,3890,2000,269227,6580,272971,360,1119
3,Bledsoe County,4560,1620,2220,610,9690,3100,1200,178860,4540,181231,160,458
4,Blount County,59550,25090,26650,6550,115820,33400,16880,3333654,59350,3381616,1190,3551
